<a href="https://colab.research.google.com/github/englissi/englissi/blob/main/Listening_Cloze_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%capture
!pip install gradio


!pip install gTTS

!pip install gtts pydub

In [16]:
import gradio as gr
import requests
from gtts import gTTS
from pydub import AudioSegment
from pydub.generators import Sine
import os

# Cloze questions
cloze_questions = [
    {
        "question": "In the small mountain village of Echo Ridge, adventure [BLANK] a part of everyday life.",
        "answer": "was"
    },
    {
        "question": "Among these children [BLANK] a bright-eyed eighth grader named Alex.",
        "answer": "was"
    },
    {
        "question": "Alex [BLANK] for his daring spirit and his love for exploring the rugged landscapes around Echo Ridge.",
        "answer": "was known"
    },
    {
        "question": "He [BLANK] a particular fascination with the old maps and tales of hidden treasures that had been lost in the mountains centuries ago.",
        "answer": "had"
    },
    {
        "question": "One day, while exploring the local library, Alex [BLANK] upon an ancient map tucked inside a forgotten book on village lore.",
        "answer": "stumbled"
    },
    {
        "question": "The map [BLANK] at the location of a lost treasure, hidden deep within a cave known as Whispering Hollow.",
        "answer": "hinted"
    },
]

# Function to create a bell sound
def create_bell_sound(filename="bell.wav"):
    # Generate a bell sound (1000 Hz tone for 500 ms)
    tone = Sine(1000).to_audio_segment(duration=500).apply_gain(-10).fade_in(50).fade_out(50)
    tone.export(filename, format="wav")

# Create the bell sound file
create_bell_sound()

# Function to convert text to speech and add bell sound
def text_to_speech_with_bell(text, filename):
    tts = gTTS(text.replace("[BLANK]", ""))
    tts.save("temp.mp3")

    # Load the generated speech and bell sound
    speech = AudioSegment.from_mp3("temp.mp3")
    bell = AudioSegment.from_wav("bell.wav")

    # Find the position of the blank and insert the bell sound
    blank_position = text.find("[BLANK]")
    part1 = text[:blank_position]
    tts_part1 = gTTS(part1).save("part1.mp3")
    speech_part1 = AudioSegment.from_mp3("part1.mp3")

    combined = speech_part1 + bell + speech[len(speech_part1):]

    # Save the final audio with the bell sound inserted
    combined.export(filename, format="mp3")
    os.remove("temp.mp3")
    os.remove("part1.mp3")
    return filename

# Generate audio files for questions
for i, question in enumerate(cloze_questions):
    audio_filename = f"question_{i+1}.mp3"
    question["audio"] = text_to_speech_with_bell(question["question"], audio_filename)

# Function to handle the cloze quiz
def cloze_quiz(state, answer):
    name, question_index, score, results = state
    question = cloze_questions[question_index]

    if answer.strip().lower() == question["answer"].lower():
        score += 1
        results.append(f"Question {question_index + 1}: Correct\n")
    else:
        results.append(f"Question {question_index + 1}: Incorrect, the correct answer is: {question['answer']}\n")

    question_index += 1

    if question_index < len(cloze_questions):
        next_question_audio = cloze_questions[question_index]["audio"]
        return (name, question_index, score, results), next_question_audio, gr.update(visible=False), gr.update(value="")
    else:
        result_text = f"* Name: {name}\n* Score: {score} out of {len(cloze_questions)}\n" + "\n".join(results)
        return (name, question_index, score, results), None, gr.update(visible=True, value=result_text), gr.update(visible=False)

# Function to start the quiz
def start_quiz(name):
    return (name, 0, 0, []), cloze_questions[0]["audio"], gr.update(visible=False), gr.update(visible=True)

# Create the Gradio interface
with gr.Blocks() as iface:
    name_input = gr.Textbox(label="Enter your name")
    start_button = gr.Button("Start Quiz")
    question_audio = gr.Audio(interactive=False)
    answer_input = gr.Textbox(label="Your Answer")
    next_button = gr.Button("Next")
    result_output = gr.Textbox(label="Results", interactive=False, visible=False)

    # Initialize the state
    state = gr.State()

    start_button.click(start_quiz, inputs=name_input, outputs=[state, question_audio, result_output, answer_input])
    next_button.click(cloze_quiz, inputs=[state, answer_input], outputs=[state, question_audio, result_output, answer_input])

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c4963494b6e96e7cee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
